In [ ]:
import sympy.physics.mechanics as me
import sympy as sm
from scipy.integrate import solve_ivp
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib
import matplotlib
import time
import pythreejs as p3js

The rotation of a rigid body around its axis of smallest inertia is known to be unstable. I want to show this
'experimentally'. As I want this to take place in 'outer space', I will set g = 0 in the
calculations.
The 'main' rotatation is about the Y axis. Small disturbances ( = rotations about the X, Z axis) will
cause the gyroscope to tumble in space.
Additionally, I want to see the accelerations an observer on the body would experience.

**Variables**
- $N$     inertial frame
- $P_0$   point fixed in the reference frame
- $A$     frame fixed to body of gyroscope
- $m_c$     mass of gyroscope, modelled as a rod
- $m_o$     mass of the observer, a particle
- $l$      length of gyroscope
- $r$      radius of gyroscope
- $i_{XX}, i_{YY}, i_{ZZ}$  moments of inertia
- $reibung$     friction

In [ ]:
start = time.time()

mc, mo, g, l, r, iXX, iYY, iZZ, reibung = sm.symbols('mc, mo, g, l, r, iXX, iYY, iZZ, reibung')
t = sm.symbols('t')

# this is used to calculate the accelerations below.
RHS = [*[sm.symbols('rhs' + str(i)) for i in range(12)]]

# Geometry
q1, q2, q3 = me.dynamicsymbols('q1, q2, q3')     # rotation of the gyroscope
u1, u2, u3 = me.dynamicsymbols('u1, u2, u3')

x, y, z = me.dynamicsymbols('x, y, z')           # coordinates of the center of gravity of the gyroscope
ux, uy, uz = me.dynamicsymbols('ux, uy, uz')
 
N = me.ReferenceFrame('N')                       # inertial frame
A = me.ReferenceFrame('A')                       # frame in which the gyroscpoe is located
P0 = me.Point('P0')
Dmc = me.Point('Dmc')                            # geometric center of gyroscope
P2 = me.Point('P2')                              # location of the observer

P0.set_vel(N, 0)

Dmc.set_pos(P0, x*N.x + y*N.y + z*N.z)           # center of mass of the gyroscope 
Dmc.set_vel(N, ux*N.x + uy*N.y + uz*N.z)

A.orient_body_fixed(N, (q1, q2, q3), '123')
rot = A.ang_vel_in(N)                            # rot, rot1 needed for kinematic equations

It is important, that in A.set_ang_vel(N, u1 * **A.x** + u2 * **A.y** + u3 * **A.z**) the frame **A** be used to define the $u_i$. 
if N was used here and further down for the kinematic equations, the mass matrix and force vector become very large: Selecting the 'wrong' frame increases the number of operations
- in MM from 314 to approximately 2,500 !
- in force from 485 to approximately 290,000 !

This is no problem in this small simulation, but it can become decisive in larger ones.

In [ ]:
A.set_ang_vel(N, u1*A.x + u2*A.y + u3*A.z)
rot1 =A.ang_vel_in(N)
                                                
P2 = Dmc.locatenew('P2', l/sm.S(2.)*A.y + r*A.z)            # Observer, mass mo
P2.v2pt_theory(Dmc, N, A)

I = me.inertia(A, iXX, iYY, iZZ)                                              
Body = me.RigidBody('Body', Dmc, A, mc, (I, Dmc))           # gyroscope
P2a = me.Particle('P2a', P2, mo)                            # observer
BODY =[Body, P2a]

**Kane's formalism**.

It is important, that in kd = [me.dot(rot-rot1, uv) for uv in **A**] + ... the frame **A** be used. If N is used, the force vector will become large. 

Note, that in acc_P2 = P2.vel(N).diff(t, **A**) one has to indicate, w.r.t. which frame one wants to differentiate. As I want to see the acceleration felt by an observer sitting on the gyroscope, I differentia w.r.t. frame A


In [ ]:
# 1. kinematic equations
kd = [me.dot(rot-rot1, uv) for uv in A] + [x.diff(t) - ux, y.diff(t) - uy, z.diff(t) - uz]

# 2. Forces, torques acting on the bodx
FL1 = [(Dmc, mc*g*N.x), (P2, mo*g*N.x)]
Torque = [(A, -reibung * sum([me.dot(A.ang_vel_in(N), uv)*uv for uv in A]))] 
FL = FL1 + Torque
       
# 3. generalized coordinates
q0 = [q1, q2, q3] + [x, y, z]
u0 = [u1, u2, u3] + [ux, uy, uz]

# 4. set up the equations
KM = me.KanesMethod(N, q_ind=q0, u_ind=u0, kd_eqs=kd)
(fr, frstar) = KM.kanes_equations(BODY, FL)

MM = KM.mass_matrix_full
print('MM dynamic symbols', me.find_dynamicsymbols(MM))
print('MM free symbols', MM.free_symbols)
print('MM contains {} operations'.format(np.sum(np.array([MM[i, j].count_ops(visual=False) 
            for i in range(MM.shape[0]) for j in range(MM.shape[1])]))), '\n')

force = KM.forcing_full
print('force dynamic symbols', me.find_dynamicsymbols(force))
print('force free symbols', force.free_symbols)
print('force contains {} operations'.format(np.sum(np.array([force[i].count_ops(visual=False) 
            for i in range(len(force))]))), '\n')

# Rotation of the gyroscope expressed in inertial frame N
rot_vel_N = [*[me.dot(A.ang_vel_in(N), uv) for uv in N]]  # get coordinates in N

# acceleration of the center of gravity as felt sitting on the gyroscope
acc_P2 = P2.vel(N).diff(t, A)
acc_dict = {i.diff(t): RHS[j] for j, i in enumerate(q0 + u0)}
acc_P2 = acc_P2.subs(acc_dict)
print('acc_P2 dynamic symbols', me.find_dynamicsymbols(acc_P2, reference_frame=A), '\n')

The sympy functions are converted to numpy functions, to be able to do numerical calculations. $cse=True$ speeds up the numerical calculations a lot. Maybe not important in this small example, but surely in bigger ones. It needs sympy 1.11.1 or better.

In [ ]:
qL = q0 + u0
pL = [mc, mo, g, l, r, iXX, iYY, iZZ, reibung]

MM_lam = sm.lambdify(qL + pL, MM, cse=True)
force_lam = sm.lambdify(qL + pL, force, cse=True)

rot_vel_N_lam = sm.lambdify(qL + pL, rot_vel_N, cse=True)
acc_P2_lam = sm.lambdify(qL + pL + RHS, [me.dot(acc_P2, uv) for uv in A], cse=True)

print("It took {:.2f} sec to establish Kane's equations".format(time.time() - start))

**Numerical Integration**

Input Data:

- $intervall$....the integration runs from 0 to intervall
- $frames$.......for the animation this number of steps per second seems to give a good animation
- $m_{c1}$..........mass of gyroscope
- $_{mo1}$..........mass of 'observer. If it is too small, numerical issues arise. I do not really know why.
- $l_1$...........length of the cylindrical gyroscope
- $r_1$...........its diameter
- $reibung$ ......friction
- $q_{11}, q_{21}, q_{31}$ ...generalized coordinate
- $u_{11}, u_{21}, u_{31}$ ...generalized angular velocities
- $x_1, y_1, z_1$ ......coordinates of the center of the gyroscope
- $ux_1, uy_1, uz_1$...the speeds

As solve_ivp has information about how it performed, I print this information.

It makes sense to give the numerical values of the symbols / dynamic symbols names similar to the names used when setting up Kane's equations, but **avoid** using the **same** ones. This will overwrite the symbols, with sometimes undesired consequences.

method = 'Radau' in solve_ivp gives 'smooth' results, but at the expense of longer running time.

In [ ]:
# Integrate numerically
start1 = time.time()

print('Arguments')
print('[mc, mo, g, l, r, iXX, iYY, iZZ, reibung]')

# Input
#========================================================
intervall = 25
frames = 30
mc1 = 1.
mo1 = 1.e-10    # if mo1 =approx 0., numerical issues arise, I assume, the mass matrix becomes singular.
l1 = 1.
r1 = 0.05
reibung1 = 0.0
disturbance = 0.0025

q11, q21, q31 = 0., 0., 0.
x1, y1, z1 = 0., 0., 0.
ux1, uy1, uz1 = 0., 0., 0.

u2 = 10.                      # main rotation around the Y axis
u1, u3 = disturbance * u2, disturbance * u2     # small disturbance (rotational speed aroud the X / Z axis)
#==========================================================
schritte = int(intervall * frames)

Iy = 0.5 * mc1 * r1**2               # from the internet for a solid cylinder
Ix = mc1/12. * (3.*r1**2 + l1**2)    # dto.

# gravity g = 0. The gyroscope is in space somewhere.
pL_vals = [mc1, mo1,  0., l1, r1, Ix, Iy, Ix, reibung1]
print(pL_vals, '\n')
print('[q1, q2, q3, x, y, z u1, u2, u3, ux, uy, uz]')

y0 = [q11, q21, q31, x1, y1, z1, u1, u2, u3, ux1 , uy1, uz1]

print(y0, '\n')

times = np.linspace(0, intervall, schritte)
t_span = (0., intervall)                        
def gradient(t, y, args):
    vals = np.concatenate((y, args))
    sol = np.linalg.solve(MM_lam(*vals), force_lam(*vals))
    return np.array(sol).T[0]

resultat1 = solve_ivp(gradient, t_span, y0, t_eval = times, args=(pL_vals,), method='Radau')
resultat = resultat1.y.T
event_dict = {-1: 'Integration failed', 0: 'Integration finished successfully', 1: 'some termination event'}
print(event_dict[resultat1.status])

print("To numerically integrate an intervall of {} sec the routine cycled {} times and it took {:.3f} sec "
      .format(intervall, resultat1.nfev, time.time() - start1), '\n')

print('resultat shape', resultat.shape, '\n')

Plot the generalized speeds, as seen in A 

In [ ]:
# numerical calculation of rhs. Would take long to do it numerically
RHS1 = np.zeros((schritte, 12))
for i in range(schritte):
     RHS1[i, :] = np.linalg.solve(MM_lam(*[resultat[i, j]for j in range(12)], *pL_vals), 
                          force_lam(*[resultat[i, j] for j in range(12)], *pL_vals)).reshape(12)

fig, ax = plt.subplots(figsize=(10,6))
for i, j in zip((6, 7, 8), ('X - coordinate', 'Y - coordinate', 'Z - coordinate')):
    ax.plot(times, resultat[:, i], label=str(j))
    ax.set_title('Rotational speed of the gyroscope as seen in A. \n As the mass of the observer is small but positive \n the rotational speeds vary a bit.')
    ax.set_ylabel('Radians/time')
ax.legend();

Plot the generalized speeds as seen by an observer at rest in N. Obviously he sees something different, the tumbling of the gyroscope.

In [ ]:
coords = np.zeros((schritte, 3))
for i in range(schritte):
    coords[i] = rot_vel_N_lam(*[resultat[i, j] for j in range(resultat.shape[1])], *pL_vals)

fig, ax = plt.subplots(figsize=(10,6))    
for i, j in enumerate(('X-Rotation in N', 'Y-Rotation in N', 'Z-Rotation in N')):
    ax.plot(times, coords[:, i], label='{}'.format(j))
ax.set_title('Rotational speed of gyroscope as seen in N. \n This is what an outside observer would see. \n This shows it is tumbling.')

ax.set_ylabel('Radians/time')
ax.legend();

Plot the acceleration felt by on observer sitting on the gyroscope

In [ ]:
coords = np.zeros((schritte, 3))
for i in range(schritte):
    coords[i] = acc_P2_lam(*[resultat[i, j] for j in range(resultat.shape[1])], *pL_vals, *[RHS1[i, k] 
                                                    for k in range(12)])

fig, ax = plt.subplots(figsize=(10,6))    
for i, j in enumerate(('X acc in A', 'Y acc in A', 'Z acc in A')):
    ax.plot(times, coords[:, i], label='{}'.format(j))
ax.set_ylabel('Acceleration of observer expressed in body frame A')
ax.set_title('Acceleration felt by an observer sitting on the gyroscope. \n As it is not constant, he concludes there is tumbling going on.')
ax.legend();

**animation using pythreejs**.

I simply copied from Jason Moore's lecture and adapted it to my needs here.

This part is to get the TA matrices. A is the coordinate system fixed to the body, to be expressed in
the inertial frame N.

TAc / TAo are 4 x 4 martices. The upper left 3 x 3 submatrix holds the position of the body, described by the body fixed frame. The location of the origin of the body is stored in the lower left 1 x 3 submatrix.

NOTE: the 'reference frame' for pythreejs seems to be:
X - axis downwards, color red
Y - axis to the right, color green (hence:)
Z - axis pointing to the observer, color blue


In [ ]:
TAc = sm.eye(4)
TAc[:3, :3] = A.dcm(N)  # this contains the information about the rotation of the body

# this contains the information about the location of the 'center' of the body
# I want the motion without the drift, present if mo != 0
TAc[3, :3] = sm.transpose(P0.pos_from(P0).to_matrix(N)) 
TAc = TAc.reshape(16, 1)

TAo = sm.eye(4)
TAo[:3, :3] = A.dcm(N)
TAo[3, :3] = sm.transpose(P2.pos_from(Dmc).to_matrix(N))
TAo = TAo.reshape(16, 1)

TA = [TAc, TAo]
TA_lam = sm.lambdify(qL + pL, TA)   # Lambdification of TA

Create the TAs, containing 'one TA' for each time step. resultat contains the results of the numeric integration, times contains the points in time, where the numeric integration was evaluated

In [ ]:
TAs = []

for k in range(resultat.shape[0]):
    zeit = times[k] 
    TAi = TA_lam(*[resultat[k, l] for l in range(resultat.shape[1])], *pL_vals)
    TAs.append([TAi[0].squeeze().tolist(), TAi[1].squeeze().tolist() ])

Define the bodies of the gyroscope and the observer

In [ ]:
body_geom_C = p3js.CylinderGeometry(radiusTop=pL_vals[4], radiusBottom=pL_vals[4], height=pL_vals[3], radialSegments=6)
body_material_C = p3js.MeshStandardMaterial(color='orange', wireframe=False)
body_mesh_C = p3js.Mesh(geometry=body_geom_C, material=body_material_C, name='cylinder')

# locate the body in 3D space and add the coordinate system of the body
body_mesh_C.matrixAutoUpdate = False
body_mesh_C.add(p3js.AxesHelper(pL_vals[3]/1.))
body_mesh_C.matrix = TAs[0][0]           # starting point of the animation

body_geom_O = p3js.SphereGeometry(radius=pL_vals[4]/2.5)
body_material_O = p3js.MeshStandardMaterial(color='blue', wireframe=False)
body_mesh_O = p3js.Mesh(geometry=body_geom_O, material=body_material_O, name='observer')

# locate the body in 3D space and add the coordinate system of the body
body_mesh_O.matrixAutoUpdate = False
body_mesh_O.add(p3js.AxesHelper(0.05))
body_mesh_O.matrix = TAs[0][1]           # starting point of the animation

Create the 'environment'. All the parameters are by trial and error, I do not really know whatthey mean.

In [ ]:
view_width = 800
view_height = 800

camera = p3js.PerspectiveCamera(position=[0., 0.0, 4.],
                                up=[-1.0, 0.0, 0.0],
                                aspect=view_width/view_height)

key_light = p3js.DirectionalLight(position=[0, 10, 10])
ambient_light = p3js.AmbientLight()

axes = p3js.AxesHelper()

children = [body_mesh_C, body_mesh_O, axes, camera, key_light, ambient_light]

scene = p3js.Scene(children=children)

controller = p3js.OrbitControls(controlling=camera)
renderer = p3js.Renderer(camera=camera, scene=scene, controls=[controller],
                         width=view_width, height=view_height)

Create the 'action'

In [ ]:
BB = [TAs[l][0] for l in range(len(TAs))]
track_C = p3js.VectorKeyframeTrack(
    name="scene/cylinder.matrix",
    times=times,
    values=BB)

BB = [TAs[l][1] for l in range(len(TAs))]
track_O = p3js.VectorKeyframeTrack(
    name="scene/observer.matrix",
    times=times,
    values=BB)

tracks = [track_C, track_O]
duration = times[-1] - times[0]
clip = p3js.AnimationClip(tracks=tracks, duration=duration)
action = p3js.AnimationAction(p3js.AnimationMixer(scene), clip, scene)
print('it took {:.3f} sec to finish the program'.format(time.time() - start))
renderer

This put the action in motion

In [ ]:
action